In [0]:
pip uninstall tensorflow

Uninstalling tensorflow-2.2.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.2.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
  Successfully uninstalled tensorflow-2.2.0


In [0]:
pip install tensorflow==1.13.2

     |████████████████████████████████| 92.6MB 63kB/s 
     |████████████████████████████████| 3.2MB 25.2MB/s 
     |████████████████████████████████| 368kB 35.8MB/s 
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0


In [0]:
pip install fasttext

     |████████████████████████████████| 71kB 4.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3016145 sha256=0defeccbda0e3e0040e468af107002583c954cecfd7b02f40f021908ae7b80c5
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [0]:
import os
from matplotlib.image import imread
import numpy as np
import pandas as pd
from tensorflow.contrib import learn
import fasttext
import numpy as np
from sklearn.utils import shuffle

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [0]:
embeddings_model = fasttext.train_unsupervised("text_corpus2.txt", model='skipgram', lr=0.04, dim=128, epoch=1)

embeddings_model.save_model("ft_skipgram_ws5_dim28.bin")

In [0]:
current_index = 0

In [0]:
dataset = pd.read_csv('Triplet_loss_dataset.csv')
dataset.dropna(how='any',inplace=True)
dataset = shuffle(dataset)
dataset = dataset.reset_index(drop = True)

In [0]:
def removing_bad_characters(test_string):
   bad_chars = [';', ':', '!', "*",'?','"',"'",'>','<','•','o','≤','â','€','¢'] 
   # remove bad_chars  
   for i in bad_chars : 
      test_string = test_string.replace(i, '') 
   return test_string

In [0]:
similar_pairs = pd.DataFrame()
similar_pairs['Possible Questions'] = dataset['Possible Questions'].str.lower()
similar_pairs['Possible Questions'] = removing_bad_characters(similar_pairs['Possible Questions'])
similar_pairs['True_Atom'] = dataset['True_Atom'].str.lower()
similar_pairs['True_Atom'] = removing_bad_characters(similar_pairs['True_Atom'])
similar_pairs['Atom'] = dataset['Atom'].str.lower()
similar_pairs['Atom'] = removing_bad_characters(similar_pairs['Atom'])

In [0]:
similar_pairs.to_csv('similar_pairs.csv')

In [0]:
triplet_list = []
for item in similar_pairs.values.tolist():
    triplet_list.append(item)
print('Triplets created successfully. # of triplets: ',len(triplet_list))

Triplets created successfully. # of triplets:  6058


In [0]:
triplet_list[1]

['what happens on the coadministraion of probenecid, methotrexate and other medicinal products with furosemide,',
 'probenecid, methotrexate and other medicinal products which, like furosemide, are extensively secreted in the renal tubules, can reduce the effect of furosemide.',
 'pharmacokinetic studies with cyclosporin have demonstrated that amlodipine does not significantly alter the pharmacokinetics of cyclosporin.']

In [0]:
triplets = triplet_list

In [0]:
input_X = [item[0] for item in triplets]
input_Y = [item[1] for item in triplets]
input_Z = [item[2] for item in triplets]

In [0]:
#word_count_per_row
wc_list_x = list(len(x.split(' ')) for x in input_X)
wc_list_y = list(len(x.split(' ')) for x in input_Y)
wc_list_z = list(len(x.split(' ')) for x in input_Z)

In [0]:
wc_list = []
wc_list.extend(wc_list_x)
wc_list.extend(wc_list_y)
wc_list.extend(wc_list_z)

In [0]:
number_of_elements = len(input_X)

In [0]:
vocab_processor = learn.preprocessing.VocabularyProcessor(max_document_length=15)

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [0]:
full_corpus = []
full_corpus.extend(input_X)
full_corpus.extend(input_Y)
full_corpus.extend(input_Z)

In [0]:
full_data = np.asarray(list(vocab_processor.fit_transform(full_corpus)))

Instructions for updating:
Please use tensorflow/transform or tf.data.


In [0]:
embeddings_lookup = []

In [0]:
for word in list(vocab_processor.vocabulary_._mapping):
    try:
        embeddings_lookup.append(embeddings_model[str(word)])
    except:
        pass

In [0]:
embeddings_lookup = np.asarray(embeddings_lookup)
np.save("embeddings_lookup.npz",embeddings_lookup)

In [0]:
def write_metadata(filename, labels):
    with open(filename, 'w') as f:
        f.write("Index\tLabel\n")
        for index, label in enumerate(labels):
            f.write("{}\t{}\n".format(index, label))
    print('Metadata file saved in {}'.format(filename))

In [0]:
vocab_processor.save('vocab')
write_metadata(os.path.join('model_triplet','metadata.tsv'),list(vocab_processor.vocabulary_._mapping))
print('Vocab processor executed and saved successfully.')

Metadata file saved in model_triplet/metadata.tsv
Vocab processor executed and saved successfully.


In [0]:
X = full_data[0:number_of_elements]
Y = full_data[number_of_elements:2*number_of_elements]
Z = full_data[2*number_of_elements:3*number_of_elements]

In [0]:
def get_triplets_batch(current_index,n):
    last_index = current_index
    current_index += n
    return X[last_index:current_index,:], Y[last_index:current_index,:],Z[last_index:current_index,:]

In [0]:
# building model
class TripletLoss:

    def __init__(self, sequence_length, vocab_size, embedding_size, filter_sizes, num_filters,output_embedding_size, dropout_keep_prob,embeddings_lookup, l2_reg_lambda=0.0):
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.output_embedding_size = output_embedding_size
        self.dropout_keep_prob = dropout_keep_prob
        self.l2_loss = tf.constant(0.0)
        self.embeddings_lookup = embeddings_lookup

    def conv_net(self, input_x, reuse=False):
        
        # Embedding layer
        with tf.variable_scope("embedding", reuse=reuse):
            self.W = tf.Variable(self.embeddings_lookup, name="W")
            self.embedded_chars = tf.nn.embedding_lookup(self.W, input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)
        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(self.filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, self.embedding_size, 1, self.num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[self.num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, self.sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = self.num_filters * len(self.filter_sizes)
        self.h_pool = tf.concat(pooled_outputs, 3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total], name="pool_flat")

        # Pre final layer / Embeddings Layer
        with tf.variable_scope("output", reuse=reuse):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, self.output_embedding_size],
                initializer=tf.contrib.layers.xavier_initializer())
            b2 = tf.Variable(tf.constant(0.1, shape=[self.output_embedding_size]), name="b2")
            self.l2_loss += tf.nn.l2_loss(W)
            self.l2_loss += tf.nn.l2_loss(b2)
            out_net = tf.nn.xw_plus_b(self.h_pool_flat, W, b2, name="scores")

        return out_net

    def triplet_loss(self, model_anchor, model_positive, model_negative, margin):
        distance1 = tf.sqrt(tf.reduce_sum(tf.pow(model_anchor - model_positive, 2), 1, keepdims=True))
        distance2 = tf.sqrt(tf.reduce_sum(tf.pow(model_anchor - model_negative, 2), 1, keepdims=True))
        return tf.reduce_mean(tf.maximum(distance1 - distance2 + margin, 0)) + 1e-9

In [0]:
# Model Hyperparameters
embedding_dim =  128
filter_sizes =  "3,4,5"
num_filters =  128
dropout_keep_prob = 0.5
l2_reg_lambda = 0.1
output_embedding_size = 128
batch_size = 100
train_iter = 100 #Total training iter
step = 20 # Save after ... iteration
learning_rate = 0.00005
max_document_length = 50 # 'maximum number of word in a sentence'
momentum = 0.99
#model
#data_src

In [0]:
import tensorflow as tf

In [0]:
model = TripletLoss(sequence_length=X.shape[1],
                vocab_size=len(vocab_processor.vocabulary_),
                embedding_size=embedding_dim,
                filter_sizes=list(map(int, filter_sizes.split(","))),
                num_filters=num_filters,
                output_embedding_size = output_embedding_size,
                dropout_keep_prob = dropout_keep_prob,
                embeddings_lookup= embeddings_lookup,
                l2_reg_lambda=l2_reg_lambda)

In [0]:
placeholder_shape = [None] + [X.shape[1]]
print("placeholder_shape", placeholder_shape)

placeholder_shape [None, 15]


In [0]:
next_batch = get_triplets_batch

In [0]:
# Setup Network
next_batch = get_triplets_batch
anchor_input = tf.placeholder(tf.int32, placeholder_shape, name='anchor_input')
positive_input = tf.placeholder(tf.int32, placeholder_shape, name='positive_input')
negative_input = tf.placeholder(tf.int32, placeholder_shape, name='negative_input')

In [0]:
margin = 3.5
anchor_output = model.conv_net(anchor_input, reuse=False)
positive_output = model.conv_net(positive_input, reuse=True)
negative_output = model.conv_net(negative_input, reuse=True)
loss = model.triplet_loss(anchor_output, positive_output, negative_output, margin)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Setup Optimizer
global_step = tf.Variable(0, trainable=False)

train_step = tf.train.MomentumOptimizer(learning_rate, momentum, 
                                        use_nesterov=True).minimize(loss,global_step=global_step)

Instructions for updating:
Use tf.cast instead.


In [0]:
from tensorflow.contrib.tensorboard.plugins import projector

In [0]:
# Start Training
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # Setup Tensorboard
    tf.summary.scalar('step', global_step)
    tf.summary.scalar('loss', loss)
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)
    merged = tf.summary.merge_all()
    writer = tf.summary.FileWriter('model_triplet', sess.graph)
    # adding embeddings to projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = "embedding/W"
    embed.metadata_path = "metadata.tsv"
    projector.visualize_embeddings(writer, config)
    print('Training...')
    # Train iter
    for i in range(train_iter):
        batch_anchor, batch_positive, batch_negative = next_batch(current_index,batch_size)
        _, l, summary_str = sess.run([train_step, loss, merged],
                                        feed_dict={anchor_input: batch_anchor, positive_input: batch_positive, negative_input: batch_negative})

        writer.add_summary(summary_str, i)
        print("\r#%d - Loss" % i, l)

        if (i + 1) % step == 0 or l <= 0.1:
            print('Saving model. Step: ',i)
            saver.save(sess, "model_triplet/model.ckpt")
        if l<=0.1:
          break
    saver.save(sess, "model_triplet/model.ckpt")
print('Training completed successfully.')

Training...
#0 - Loss 3.2180228
#1 - Loss 3.2025485
#2 - Loss 3.1794517
#3 - Loss 3.1488092
#4 - Loss 3.110696
#5 - Loss 3.0651877
#6 - Loss 3.0123577
#7 - Loss 2.9522796
#8 - Loss 2.8850362
#9 - Loss 2.8106997
#10 - Loss 2.729361
#11 - Loss 2.6410973
#12 - Loss 2.5459907
#13 - Loss 2.4441242
#14 - Loss 2.3355913
#15 - Loss 2.2204893
#16 - Loss 2.0989213
#17 - Loss 1.9709837
#18 - Loss 1.836784
#19 - Loss 1.6964422
Saving model. Step:  19
#20 - Loss 1.5500811
#21 - Loss 1.397844
#22 - Loss 1.239871
#23 - Loss 1.0780437
#24 - Loss 0.9150214
#25 - Loss 0.7548722
#26 - Loss 0.6015364
#27 - Loss 0.47006044
#28 - Loss 0.35935864
#29 - Loss 0.26355964
#30 - Loss 0.18733245
#31 - Loss 0.12155407
#32 - Loss 0.07108684
Saving model. Step:  32
Training completed successfully.
